<a href="https://colab.research.google.com/github/icecore013/image-segmentation-keras/blob/master/1b_patches_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import and configuration variables

In [ ]:
# Loading of our drive
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'

Mounted at /content/gdrive


In [ ]:
# Import
import os
from os import path as op
import random

!pip install patchify
from patchify import patchify

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

!pip install rasterio
import rasterio

import tensorflow as tf
!pip install keras

from sklearn.model_selection import train_test_split

!pip install tifffile
import tifffile

from tqdm import tqdm

     |████████████████████████████████| 19.3 MB 257 kB/s 


In [ ]:
# Config variables

CHECK_DIRECTORY = '/content/gdrive/MyDrive/Data/Training_data/CAL_database'
#CHECK_DIRECTORY = '/content/gdrive/MyDrive/Fires/PERIMETRES/PERIM_DATABASE/CAL_database'


ROOT_DIRECTORY = '/content/gdrive/MyDrive/Data/Training_data'
PATCH_DIRECTORY = '/content/gdrive/MyDrive/Data/Training_data/Final_patches'


PREDICTION_THRESHOLD = 0.4

IMAGE_SCALING_FACTOR = 10000
MASK_SCALING_FACTOR = 255

PATCH_SIZE = 256

IMG_CHANNELS = 5
MASK_CHANNELS = 1

BATCH_SIZE = 32

# Visual check of the complete image and corresponding mask

In [ ]:
# Listing all filenames
total_Flist = sorted(os.listdir(op.join(CHECK_DIRECTORY, 'images' )))
total_Flist = [i for i in total_Flist if i.endswith('.tif')]
total_Flist = [i for i in total_Flist if 'post' in i]

# Initialisation of the image dataset
image_dataset = []

for i, filename in enumerate(total_Flist):
    # Lecture du fichier tiff et conversion en Array
    with rasterio.open(op.join(CHECK_DIRECTORY, 'images', filename ), 'r') as ds:
      arr = ds.read()

    # Normalize the data (divide par 10000)
    arr = arr / 10000

    # Change dimension order (number of channel at the end)
    arr = np.moveaxis(arr, 0, -1)

    # Check the dimensions of the Array
    print(arr.shape, i, filename)

    # Conversion Array en Tensor
    tf_image = tf.convert_to_tensor(arr, dtype = tf.float32)

    # Resize the image
    tf_image_resized = tf.image.resize(tf_image, size=(PATCH_SIZE, PATCH_SIZE))

    # Ajout du Train Dataset
    image_dataset.append(tf_image_resized)

image_dataset = np.array(image_dataset)
print('Nb of images: ' + str(image_dataset.shape[0]))

(1433, 761, 5) 0 CAL_database_Sentinel2_120_postFire_RGBIR.tif
(902, 1132, 5) 1 CAL_database_Sentinel2_140_postFire_RGBIR.tif
(331, 603, 5) 2 CAL_database_Sentinel2_155_postFire_RGBIR.tif
(1297, 1195, 5) 3 CAL_database_Sentinel2_164_postFire_RGBIR.tif
(4379, 3718, 5) 4 CAL_database_Sentinel2_176_postFire_RGBIR.tif
(1771, 1682, 5) 5 CAL_database_Sentinel2_201_postFire_RGBIR.tif
(996, 1029, 5) 6 CAL_database_Sentinel2_225_postFire_RGBIR.tif
(3631, 2658, 5) 7 CAL_database_Sentinel2_226_postFire_RGBIR.tif
(1347, 995, 5) 8 CAL_database_Sentinel2_227_postFire_RGBIR.tif
(980, 968, 5) 9 CAL_database_Sentinel2_238_postFire_RGBIR.tif
(1514, 1954, 5) 10 CAL_database_Sentinel2_240_postFire_RGBIR.tif
(3062, 3268, 5) 11 CAL_database_Sentinel2_243_postFire_RGBIR.tif
(747, 974, 5) 12 CAL_database_Sentinel2_270_postFire_RGBIR.tif
(502, 878, 5) 13 CAL_database_Sentinel2_272_postFire_RGBIR.tif
(488, 458, 5) 14 CAL_database_Sentinel2_293_postFire_RGBIR.tif
(1194, 1291, 5) 15 CAL_database_Sentinel2_302_pos

In [ ]:
# Listing all mask filenames
mask_Flist = sorted(os.listdir(op.join(CHECK_DIRECTORY, 'masks' )))
mask_Flist = [i for i in mask_Flist if i.endswith('.tif')]

# Initialisation of the mask dataset
mask_dataset = []

for i, filename in enumerate(mask_Flist):
    # tif file opening and conversion in Array
    with rasterio.open(op.join(CHECK_DIRECTORY, 'masks', filename ), 'r') as ds:
      arr = ds.read()

    # Normalize the data (divide par 255)
    arr = arr / 255

    # Change dimension order (number of channel at the end)
    arr = np.moveaxis(arr, 0, -1)

    # Check the dimensions of the Array
    print(arr.shape, i, filename)

    # Conversion Array en Tensor
    tf_label = tf.convert_to_tensor(arr, dtype = tf.int32)

    # Resize the image
    tf_label_resized = tf.image.resize(tf_label, size=(PATCH_SIZE, PATCH_SIZE))

    # Ajout du Train Dataset
    mask_dataset.append(tf_label_resized.numpy())

mask_dataset = np.array(mask_dataset)
print('Nb of masks: ' + str(image_dataset.shape[0]))

(1433, 761, 1) 0 CAL_database_Sentinel2_120_postFire_label.tif
(902, 1132, 1) 1 CAL_database_Sentinel2_140_postFire_label.tif
(331, 603, 1) 2 CAL_database_Sentinel2_155_postFire_label.tif
(1297, 1195, 1) 3 CAL_database_Sentinel2_164_postFire_label.tif
(4379, 3718, 1) 4 CAL_database_Sentinel2_176_postFire_label.tif
(1771, 1682, 1) 5 CAL_database_Sentinel2_201_postFire_label.tif
(996, 1029, 1) 6 CAL_database_Sentinel2_225_postFire_label.tif
(3631, 2658, 1) 7 CAL_database_Sentinel2_226_postFire_label.tif
(1347, 995, 1) 8 CAL_database_Sentinel2_227_postFire_label.tif
(980, 968, 1) 9 CAL_database_Sentinel2_238_postFire_label.tif
(1514, 1954, 1) 10 CAL_database_Sentinel2_240_postFire_label.tif
(3062, 3268, 1) 11 CAL_database_Sentinel2_243_postFire_label.tif
(747, 974, 1) 12 CAL_database_Sentinel2_270_postFire_label.tif
(502, 878, 1) 13 CAL_database_Sentinel2_272_postFire_label.tif
(488, 458, 1) 14 CAL_database_Sentinel2_293_postFire_label.tif
(1194, 1291, 1) 15 CAL_database_Sentinel2_302_pos

In [ ]:
def visual_check(image, label):

    # Selection of the first 3 channels
    image_rgb = image[:, :, :3]

    # Reorder for RGB image:
    #   Channels from GEE are: 2(B), 3(G), 4(R)
    #   Expected channels for RGB image: 4(R), 3(G), 2(B)
    image_rgb = image_rgb[:, :, ::-1] * 5

    # Finding the mask in mask_dataset    
    label_2D = label[:, :, 0]

    # Display graphs
    plt.figure(figsize=(20, 20))
    plt.subplot(1, 2, 1)
    plt.imshow(image_rgb, vmin=0)
    plt.title("Sentinel 2 RGB image")

    plt.subplot(1, 2, 2)
    plt.imshow(label_2D, cmap='gray', vmin=0, vmax=1)
    plt.title(("Label image"))

    plt.show()

# Checking some images visually to find unusable images (snow, smoke, clouds...)
for i in range(0, len(image_dataset)):
    print(i, total_Flist[i])
    print(i, mask_Flist[i])
    image = image_dataset[i]
    label = mask_dataset[i]
    visual_check(image, label)

# Generating patches

In [ ]:
# Listing the relevant filenames

image_filenames = []
mask_filenames = []

for path, subdirs, files in os.walk(ROOT_DIRECTORY):
    dirname = path.split(os.path.sep)[-1]
    if dirname == 'images':
        images = sorted(os.listdir(path))
        for i, image_name in enumerate(images):
            if 'post' in image_name:                    # Only post fire images
                if image_name.endswith(".tif"):         # Only tif images
                    image_filenames.append(op.join(path, image_name))

for path, subdirs, files in os.walk(ROOT_DIRECTORY):
    dirname = path.split(os.path.sep)[-1]
    if dirname == 'masks':
        masks = sorted(os.listdir(path))
        for i, mask_name in enumerate(masks):
            if 'post' in mask_name:                    # Only post fire images
                if mask_name.endswith(".tif"):         # Only tif images
                    mask_filenames.append(op.join(path, mask_name))

In [ ]:
def patchify_images(large_image_path, NB_CHANNELS, target_folder, scaling_factor):

  image_dataset = []
  with rasterio.open(large_image_path, 'r') as ds:        # Open the file using rasterio
    arr = ds.read()
  arr = arr / scaling_factor           # Normalize the data
  arr = np.moveaxis(arr, 0, -1)        # Change dimension order (number of channel at the end)

  if arr.shape[0] > PATCH_SIZE and arr.shape[1] > PATCH_SIZE:       # Only big enough images
      
      SIZE_X = (arr.shape[0]//PATCH_SIZE)*PATCH_SIZE                # Nearest size divisible by our patch size
      SIZE_Y = (arr.shape[1]//PATCH_SIZE)*PATCH_SIZE                # Nearest size divisible by our patch size
      image = arr[:SIZE_X, :SIZE_Y, :]                              # Crop the images to get a multiple of patches

      print("Now patchifying image: ", large_image_path)               # Extract patches from each image
      patches_img = patchify(image, (PATCH_SIZE, PATCH_SIZE, NB_CHANNELS), step=PATCH_SIZE)  # Step=256 for 256 patches means no overlap

      for i in range(patches_img.shape[0]):
          for j in range(patches_img.shape[1]):
              
              single_patch_img = patches_img[i,j,:,:,:,:]
              single_patch_img = single_patch_img[0]                # Drop the extra unecessary dimension that Patchify adds.

              single_patch_filename = op.join(ROOT_DIRECTORY, 'Final_patches', target_folder, large_image_path.split('/')[8].split('.')[0] + '_' + str(i) + '_' + str(j) + '.tif')
              tifffile.imsave(single_patch_filename, single_patch_img)

  return

In [ ]:
# Image patches generation
for image_filename in tqdm(image_filenames):
  patchify_images(image_filename, IMG_CHANNELS, 'patchified_images', 10000)

print('Number of generated image patches: '+ str(len(os.listdir(op.join(PATCH_DIRECTORY, 'patchified_images')))))

  3%|▎         | 1/31 [00:00<00:09,  3.24it/s]

Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/images/CAL_database_Sentinel2_120_postFire_RGBIR.tif


  6%|▋         | 2/31 [00:00<00:09,  3.13it/s]

Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/images/CAL_database_Sentinel2_140_postFire_RGBIR.tif
Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/images/CAL_database_Sentinel2_155_postFire_RGBIR.tif
Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/images/CAL_database_Sentinel2_164_postFire_RGBIR.tif


 13%|█▎        | 4/31 [00:01<00:09,  2.73it/s]

...

Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/images/CAL_database_Sentinel2_8326_postFire_RGBIR.tif
Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/images/CAL_database_Sentinel2_8351_postFire_RGBIR.tif


 97%|█████████▋| 30/31 [00:29<00:00,  2.79it/s]

Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/images/CAL_database_Sentinel2_8438_postFire_RGBIR.tif
Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/images/CAL_database_Sentinel2_8578_postFire_RGBIR.tif


100%|██████████| 31/31 [00:29<00:00,  1.05it/s]

Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/images/CAL_database_Sentinel2_8630_postFire_RGBIR.tif
Number of generated image patches: 806


In [ ]:

# Mask patches generation
for mask_filename in tqdm(mask_filenames):
  patchify_images(mask_filename, MASK_CHANNELS, 'patchified_masks', 255)

print('Number of generated mask patches: '+ str(len(os.listdir(op.join(PATCH_DIRECTORY, 'patchified_masks')))))

  0%|          | 0/31 [00:00<?, ?it/s]

Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_120_postFire_label.tif


  3%|▎         | 1/31 [00:00<00:05,  5.92it/s]

Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_140_postFire_label.tif


  6%|▋         | 2/31 [00:00<00:13,  2.16it/s]

...

Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_40_postFire_label.tif
Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_41_postFire_label.tif
Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_420_postFire_label.tif
Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_42_postFire_label.tif
Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_8261_postFire_label.tif


 84%|████████▍ | 26/31 [00:09<00:00,  7.58it/s]

Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_8326_postFire_label.tif
Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_8351_postFire_label.tif


100%|██████████| 31/31 [00:09<00:00,  3.12it/s]

Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_8438_postFire_label.tif
Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_8578_postFire_label.tif
Now patchifying image:  /content/gdrive/MyDrive/Data/Training_data/CAL_database/masks/CAL_database_Sentinel2_8630_postFire_label.tif
Number of generated mask patches: 806
